In [223]:
import requests
import numpy as np
import lxml.html
import re
import pandas as pd
from lxml.etree import tostring
link = "http://notelections.online"
tec = "Территориальная избирательная комиссия"
precinct = "УИК"
left_table_id = "Сумма"
pattern = "\d+"
get_count_from = np.array([13,14,15])
digital_tec_check = "Число избирателей"

In [224]:
def get_table_by_first_row_text(text, page):
    tree = lxml.html.fromstring(page.text)
    tables = tree.xpath("//table")
    for table in tables:
        rows = table.getchildren()
        for first_row_cell in rows[0].xpath("td/nobr/a | td/nobr/b | td"):
            if (first_row_cell.text) and (text in first_row_cell.text):
                return table
           

In [225]:
main_page = requests.get("http://notelections.online/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222")

In [226]:
tec_links_table = get_table_by_first_row_text(tec, main_page)
main_table = pd.DataFrame()
for link_cell in tec_links_table.xpath("tr/td/nobr/a")[:-1]:
    tec_number = re.findall(pattern, link_cell.text)
    tec_id = int(tec_number[0]) if tec_number else 999999
    
    tec_link = link + link_cell.attrib['href']
    tec_page = requests.get(tec_link)
    tec_table_tree = get_table_by_first_row_text(precinct, tec_page)
    
    tec_table = pd.read_html(tostring(tec_table_tree))[0].T
    
    tec_table[12] = tec_table[12].fillna(tec_id)
    tec_table.index = tec_table[0].apply(lambda x: re.findall(pattern, x)[0])
    tec_table.drop([0], axis = 1, inplace=True)
    for column in get_count_from:
        tec_table[column] = tec_table[column].apply(lambda x: int(x.split(" ")[0]))
    column_names = pd.read_html(tostring(get_table_by_first_row_text(left_table_id, tec_page)))[0].iloc[1:, 1].values
    column_names[11] = "Tec_number"
    tec_table.columns = column_names
    main_table = main_table.append(tec_table)

In [229]:
tec_link = link + tec_links_table.xpath("tr/td/nobr/a")[-1].attrib['href']
tec_page = requests.get(tec_link)
tec_table_tree = get_table_by_first_row_text(digital_tec_check, tec_page)
tec_table = pd.read_html(tostring(tec_table_tree))[0].iloc[:, -1].T
tec_table = tec_table.to_frame().T
for column in (get_count_from-1):
        tec_table[column] = tec_table[column].apply(lambda x: int(x.split(" ")[0]))
tec_table[11] = tec_table[11].fillna(99999)
tec_table.columns = column_names
tec_table.index = [99999]

In [230]:
main_table = main_table.append(tec_table)

In [246]:
column_names = ["All", "get_by_commission", "issued_indoors", "issued_outdoors",
                "not_used", "in_portable_box", "in_stationary_box", "not_valid",
                "valid", "losted", "not_taken_in_count", "tec_number",
                "Amosov", "Beglov", "Tihonova"]
main_table.columns = column_names
main_table = main_table.transform(lambda x: x.astype('int'))

In [250]:
main_table.to_csv("voting_spb_governor_2019.csv")